In [1]:
from ursina import *
from ursina.prefabs.first_person_controller import FirstPersonController #simulador de una persona

#grass_texture= load_texture('imagenes/grass_block.png')
#stone_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/stone_block.png')
#brick_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/brick_block.png')
#dirt_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/dirt_block.png')
class Voxel(Button):
    def __init__(self,position=(0,0,0)):
        super().__init__(
            parent=scene,
            position=position,
            model='cube',
            origin_y=0.5,
            texture='white_cube',
            color=color.white,
            highlight_color=color.lime
            
        )
    def input(self,key):  #elevaciones de cajones
        if self.hovered:
            if key == 'left mouse down':
                voxel=Voxel(position=self.position+mouse.normal)
            if key == 'right mouse down':
                destroy(self)


ursina version: 3.4.0
package_folder: /home/RobertoPacho/anaconda3/lib/python3.8/site-packages/ursina
asset_folder: /home/RobertoPacho/anaconda3/lib/python3.8/site-packages
blender_paths:
{}
OS: posix
screen resolution: (1920, 1080)


In [2]:
from neo4j import GraphDatabase
from tkinter import *
import tkinter.font as tkFont
cuenta=0

root = Tk()
root.title("LOGIN IA")
root.geometry('400x400')
root.resizable(width=0, height=0)
fontStyle = tkFont.Font(family="Lucida Grande", size=12)
fontStyle1 = tkFont.Font(family="Lucida Grande", size=30)
fontStyle2 = tkFont.Font(family="Lucida Grande", size=15)

eusuario=Label(root,text='LOGIN', font=fontStyle1)
eusuario.place(x=150,y=20)

eusuario=Label(root,text='Usuario', font=fontStyle)
eusuario.place(x=50,y=100)
usuario = Entry(root, fg='black', bg='white', justify='center')
usuario.place(x=110, y=100, width=200)           

eedad=Label(root,text='Edad: ', font=fontStyle)
eedad.place(x=50,y=150)
edad = Entry(root, fg='black', bg='white', justify='center')
edad.place(x=110, y=150, width=100)

epais=Label(root,text='Ciudad/Pais: ', font=fontStyle)
epais.place(x=50,y=200)
pais = Entry(root, fg='black', bg='white', justify='center')
pais.place(x=110, y=200, width=200)

ecorreo=Label(root,text='correo', font=fontStyle)
ecorreo.place(x=50,y=250)
correo = Entry(root, fg='black', bg='white', justify='center')
correo.place(x=110, y=250, width=200) 

class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close() 
        
    def crear_usuario(self, tx, nombre):
        tx.run("CREATE (Usuario:Jugador {usuario: $nombre})", nombre=nombre)
        
    def crear_usuarioj(self, tx, usr):
        tx.run("CREATE (Usr:Jugador {nombre: $usr})", usr=usr)
    
    def crear_edad(self, tx, edad):
        tx.run("CREATE (Edad:Jugador {edad: $edad})", edad=edad)
        
    def crear_pais(self, tx, pais):
        tx.run("CREATE (Pais:Jugador {pais: $pais})", pais=pais)

    def crear_correo(self, tx, correo):
        tx.run("CREATE (Correo:Jugador {correo: $correo})", correo=correo)
    
    def crear_puntaje(self, tx, puntaje):
        tx.run("CREATE (Puntaje:Jugador {puntaje: $puntaje})", puntaje=puntaje)
        
        
    def crear_relacion_usuario_usr(self, tx, usuario, usr):
        tx.run("MATCH (a:Jugador {usuario: $usuario}) "
               "MATCH (b:Jugador {nombre: $usr}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, usr=usr)    
        
    def crear_relacion_usuario_edad(self, tx, usuario, edad):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {edad: $edad}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, edad=edad)
    
    def crear_relacion_usuario_pais(self, tx, usuario, pais):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {pais: $pais}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, pais=pais)
        
    def crear_relacion_usuario_correo(self, tx, usuario, correo):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {correo: $correo}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, correo=correo)
    
    def crear_relacion_usuario_puntaje(self, tx, usuario, puntaje):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {puntaje: $puntaje}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, puntaje=puntaje)
        
    def cercanos(self, tx):
        result=tx.run("CALL gds.beta.knn.stream('vecinos', {\n"
                    "topK: 1,\n"
                    "nodeWeightProperty: 'puntaje',\n"
                    "randomSeed: 42,\n"
                    "concurrency: 1,\n"
                    "sampleRate: 1.0,\n"
                    "deltaThreshold: 0.0\n"
                "})\n"
                "YIELD node1, node2, similarity\n"
                "RETURN gds.util.asNode(node1).puntaje AS Person1, gds.util.asNode(node2).puntaje AS Person2, similarity\n"
                "ORDER BY similarity DESCENDING, Person1, Person2")
        for line in result:
            r1=(line["similarity"])
            r2=(line["Person1"])
            r3=(line["Person2"])
            print(r1,"   ",r2,"   ",r3)
            
def contador():
    global cuenta
    cuenta += 1    
    
def vecinoscercanos():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'proyecto')
    with neo4j._driver.session() as session:
        session.read_transaction(neo4j.cercanos)
        
def jugar():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'proyecto')
    usr=str(usuario.get())
    ed=str(edad.get())
    pa=str(pais.get())
    mail=str(correo.get())
    
    with neo4j._driver.session() as session:

        session.write_transaction(neo4j.crear_usuarioj , usr)
    session.write_transaction(neo4j.crear_edad , ed)
    session.write_transaction(neo4j.crear_pais , pa)
    session.write_transaction(neo4j.crear_correo , mail)

    session.write_transaction(neo4j.crear_relacion_usuario_usr,"Usuario",usr)
    session.write_transaction(neo4j.crear_relacion_usuario_edad,usr,ed)
    session.write_transaction(neo4j.crear_relacion_usuario_pais,usr,pa)
    session.write_transaction(neo4j.crear_relacion_usuario_correo,usr,mail)
    
    app= Ursina()
    for z in range(40):
        for x in range(40):
            voxel=Voxel(position=(x,0,z))
            contador()
    
        session.write_transaction(neo4j.crear_puntaje , cuenta)
        session.write_transaction(neo4j.crear_relacion_usuario_puntaje,usr,cuenta)
    
    player=FirstPersonController()
    app.run()
    
btn1 = Button(root, text="Ingresar", bg="MediumAquamarine", fg="white", height = 2, width = 15,font=fontStyle2,command=jugar)
btn1.place(x=120, y=300)
root.mainloop()

#/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/grass_block.png


In [3]:
vecinoscercanos()

1.0     40     40
1.0     40     40
1.0     80     80
1.0     80     80
1.0     200     200
1.0     200     200
1.0     240     240
1.0     240     240
1.0     320     320
1.0     320     320
1.0     360     360
1.0     360     360
1.0     400     400
1.0     400     400
1.0     440     440
1.0     440     440
1.0     480     480
1.0     480     480
1.0     520     520
1.0     520     520
1.0     560     560
1.0     560     560
1.0     600     600
1.0     600     600
1.0     640     640
1.0     640     640
1.0     680     680
1.0     680     680
1.0     720     720
1.0     720     720
1.0     760     760
1.0     760     760
1.0     800     800
1.0     800     800
1.0     840     840
1.0     840     840
1.0     880     880
1.0     880     880
1.0     920     920
1.0     960     960
1.0     960     960
1.0     1000     1000
1.0     1000     1000
1.0     1040     1040
1.0     1040     1040
1.0     1120     1120
1.0     1120     1120
1.0     1160     1160
1.0     1160     1160
1.0     1200